In [121]:
at = 'sfj a fsaf kd'
a = ''
for j in range(3):
    at = ' '.join(at.split())
    a = a+' '+at
a

' sfj a fsaf kd sfj a fsaf kd sfj a fsaf kd'

In [33]:
import numpy as np
from numpy import array
import pickle as plk
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import Dropout
from keras.layers import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
import os
from sacrebleu import sentence_bleu

In [2]:
# df = pd.read_excel('/Users/luyin/Desktop/project/Q&A.xlsx',header = 0)
df = pd.read_excel(os.getcwd()+'/Q&A_Database_new.xlsx','QA', skiprows=3)

In [3]:
l = df['Breakout'].unique() # 79 unique analyst
dic = {} #create dictionary for questions
for category in l:
    list_ = list(df.loc[df['Breakout']  == category]['Question'])
    dic[category] = list_

In [4]:
import spacy
import re
import string
# Load English tokenizer, tagger, parser, NER and word vectors

# tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

#tokenize sentence by sentence
def question_split(input_):
    list_ = []
    for q in input_:
        q = q.split('\n')
        if len(q) ==1:
            list_.append(q)
    return list_

def tokenize(sent):
#   sent = re.sub('[^A-Za-z&]', ' ', sent) # replace non-letter with space
#   sent = re.sub(r'\b[a-zA-Z]\b', '', sent) #remove single letter 
    sent = re.sub('y ou', 'you', sent)
    sent = re.sub('y es', 'yes', sent)
    sent = re.sub('v o', 'vo', sent)
    sent = re.sub("don't", 'dont', sent)
    sent = re.sub('[^A-Za-z&]', ' ', sent)
#     tokens = tokenizer(sent)
    return sent.split()

# tokens = tokenize(" going to hit them one way or another strong dollar did seem to have a huge impact and y ou, what are you doing? I'm 's what do you and me think or like apples and Apple is looking at buying and bought U.K. startup for $1 billion. '\n' another sentence")
# for token in tokens:
#     print (token)

In [5]:
df['Breakout'].unique()

array(['CIB', 'Capital', 'Balance sheet', 'Expenses', 'CB',
       'Credit costs', 'Accounting and taxes', 'CCB', 'Other topics',
       'AWM', 'Legal', 'Macroeconomic update', 'Revenue',
       'Regulatory topics', 'Capital '], dtype=object)

In [6]:
def tokenize_analyst_q(name):
    all_tokens = []
    for q in dic[name]:
        tokens = tokenize(q)
        all_tokens += tokens
    return all_tokens

In [7]:
#load training and validation dataset
dic_len = len(dic['Balance sheet'])
all_data = dic['Balance sheet']
train_data = dic['Balance sheet'][:500]
val_data = dic['Balance sheet'][500: dic_len]

In [8]:
all_tokens = []
for q in dic['Balance sheet']:
    tokens = tokenize(q)
    all_tokens += tokens

In [9]:
def build_vocab(input_):
    vocab = sorted(set(input_))
    vocab_to_int = dict((c, i) for i, c in enumerate(vocab))
    int_to_vocab = dict((i, c) for i, c in enumerate(vocab))
    n_total = len(input_)
    n_vocab = len(vocab)
    print ("Total Words: {}".format(n_total))
    print ("Total Vocab: {}".format(n_vocab))
    return n_total, n_vocab

In [18]:
# organize into sequences of tokens
def build_sequence(length, input_):
    sequences = list()
    for l in range(length, len(input_)):
        seq = input_[l-length:l]
        line = ' '.join(seq)
        sequences.append(line)
#     print('Total Sequences: {}' .format(len(sequences)))
    return sequences

In [19]:
def data_generator(dic, length):
    out_list = []
    raw = question_split(dic)
    for i in range(len(raw)):
        sent = tokenize(raw[i][0].lower())
        q_sequence = build_sequence(length, sent)
        out_list += q_sequence
    return out_list

In [20]:
all_d = data_generator(all_data,20)
train_d = data_generator(train_data,20)
val_d = data_generator(val_data,20)

In [13]:
# def save_file(lines, filename):
#     data = '\n'.join(lines)
#     file = open(filename, 'w')
#     file.write(data)
#     file.close()

# out_filename = 'Balance sheet.txt'
# save_file(all_d, out_filename)


In [14]:
# load doc into memory
# def load_file(filename):
#     file = open(filename, 'r')
#     text = file.read()
#     file.close()
#     return text
 
# in_filename = 'Balance sheet.txt'
# doc = load_file(in_filename)
# lines = doc.split('\n')

In [22]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(lines) #fit on texts
# sequences = tokenizer.texts_to_sequences(lines)
# vocab_size = len(tokenizer.word_index) + 1
# sequences = array(sequences)
# X, y = sequences[:,:-1], sequences[:,-1]
# y = to_categorical(y, num_classes=vocab_size)

In [41]:
words_to_load = 50000
PAD_IDX = 0
UNK_IDX = 1
SOS_IDX = 2
EOS_IDX = 3
import numpy as np
# reserve the 1st 2nd token for padding and <UNK> respectively
with open('/Users/cyian/Desktop/NYU/FALL2018/DS-GA1011_NLP/hws/HW2/wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft_en = np.zeros((words_to_load+4, 300))
    words_ft_en = {}
    idx2words_ft_en = {}
    ordered_words_ft_en = []
    ordered_words_ft_en.extend(['<pad>', '<unk>', '<s>', '</s>'])
    loaded_embeddings_ft_en[0,:] = np.zeros(300)
    loaded_embeddings_ft_en[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[2,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[3,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft_en[i+4, :] = np.asarray(s[1:])
        words_ft_en[s[0]] = i+4
        idx2words_ft_en[i+4] = s[0]
        ordered_words_ft_en.append(s[0])
    words_ft_en['<pad>'] = PAD_IDX
    words_ft_en['<unk>'] = UNK_IDX
    words_ft_en['<s>'] = SOS_IDX
    words_ft_en['</s>'] = EOS_IDX
    idx2words_ft_en[PAD_IDX] = '<pad>'
    idx2words_ft_en[UNK_IDX] = '<unk>'
    idx2words_ft_en[SOS_IDX] = '<s>'
    idx2words_ft_en[EOS_IDX] = '</s>'

In [21]:
def word_to_id(word_list):
    return [words_ft[x] if x in ordered_words_ft else UNK_IDX for x in word_list.split()]
def sent_to_id(sent_list):
    return [word_to_id(x) for x in sent_list]

In [55]:
# len_list = []
# for x in lines:
#     len_list.append(len(x.split()))

In [56]:
sorted(len_list)[-1]

20

In [26]:
import nltk
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 20

In [45]:
def keras_rnn_predict(samples, empty=SOS_IDX, rnn_model=model, maxlen=MAX_SEQUENCE_LENGTH):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty)
    return rnn_model.predict(data, verbose=0)

def beamsearch(predict = keras_rnn_predict,
               k=1, maxsample=400, use_unk=False, oov=UNK_IDX, empty=SOS_IDX, eos=EOS_IDX):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    
    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [[empty]]
    live_scores = [0]

    while live_k and dead_k < k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        cand_flat = cand_scores.flatten()

        # find the best (lowest) scores we have from all possible samples and new words
        ranks_flat = cand_flat.argsort()[:(k-dead_k)]
        live_scores = cand_flat[ranks_flat]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_flat]

        # live samples that should be dead are...
        zombie = [s[-1] == eos or len(s) >= maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]  # remove first label == empty
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores   

In [59]:
len(train_d)

16037

In [63]:
print(len(train_d[0].split()))

20


In [55]:
train_label.shape

(16037, 3146)

In [57]:
train_input.shape

(16037, 19)

In [84]:
len(train_input)

16037

In [87]:
len(train_input[0])

19

In [90]:
MAX_SEQUENCE_LENGTH

20

In [104]:
train_label.shape

(1, 16037)

In [116]:
train_label.shape

(16037, 300)

In [117]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens) #fit on texts
def data_loader(data_input):
    X = []
    y = []
    for i in range(len(data_input)):
        X.append([words_ft_en[x] if x in ordered_words_ft_en else UNK_IDX for x in data_input[i].split()[:-1]])
        sub = data_input[i].split()[-1]
        if sub in ordered_words_ft_en:
            y.append(loaded_embeddings_ft_en[words_ft_en[sub]])
        else:
            y.append(loaded_embeddings_ft_en[UNK_IDX])
    return np.array(X), np.array(y)
train_input, train_label =  data_loader(train_d)
val_input, val_label =  data_loader(val_d)
# sequences = tokenizer.texts_to_sequences(lines)
# vocab_size = len(tokenizer.word_index) + 1
# MAX_NUM_WORDS = words_to_load
# tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
# tokenizer.fit_on_texts(lines)
# sequences = tokenizer.texts_to_sequences(lines)
# word_index = tokenizer.word_index
# data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# X, y = data[:,:-1], data[:,-1]
vocab_size = words_to_load+4
# y = to_categorical(y, num_classes=vocab_size)
# seq_length = X.shape[1]
 
# define model
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[loaded_embeddings_ft_en],
                            input_length=MAX_SEQUENCE_LENGTH-1,
                            trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(100, return_sequences=True)))
model.add(Bidirectional(GRU(100)))
model.add(Dense(100, activation='relu'))
model.add(Dense(train_label.shape[1], activation='softmax'))

#import the checkpoint to save current model
filepath="GRU_1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
callbacks_list = [checkpoint, earlystopper]
# compile model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# fit the model
model.fit(train_input, train_label, validation_data =(val_input, val_label), batch_size= 200, epochs=100, callbacks=callbacks_list)

 
# save the model to file
model.save('model.h5')
# save the tokenizer
plk.dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 19, 300)           15001200  
_________________________________________________________________
bidirectional_33 (Bidirectio (None, 19, 200)           240600    
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 200)               180600    
_________________________________________________________________
dense_32 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_33 (Dense)             (None, 300)               30300     
Total params: 15,472,800
Trainable params: 471,600
Non-trainable params: 15,001,200
_________________________________________________________________
Train on 16037 samples, validate on 6896 samples
Epoch 1/100
16037/16037 [==============================] - 26s 2ms/step - l

In [47]:
# Define the layers
from keras.models import Model
from keras.layers import Dense, Input, GRU
from keras.layers.embeddings import Embedding
word_dim = 300
num_tokens = words_to_load+4
word_vec_input = Input(shape=(word_dim,))
decoder_inputs = Input(shape=(None,))
decoder_embed = Embedding(input_dim=num_tokens, output_dim=word_dim, mask_zero=True)
decoder_gru_1 = GRU(word_dim, return_sequences=True, return_state=False)
decoder_gru_2 = GRU(word_dim, return_sequences=True, return_state=True)
decoder_dense = Dense(num_tokens, activation='softmax')

# Connect the layers
embedded = decoder_embed(decoder_inputs)
gru_1_output = decoder_gru_1(embedded, initial_state=word_vec_input)
gru_2_output, state_h = decoder_gru_2(gru_1_output)
decoder_outputs = decoder_dense(gru_2_output)

# Define the model that will be used for training
training_model = Model([word_vec_input, decoder_inputs], decoder_outputs)

# Also create a model for inference (this returns the GRU state)
decoder_model = Model([word_vec_input, decoder_inputs], [decoder_outputs, state_h])

In [49]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    15001200    input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
gru_3 (GRU)                     (None, None, 300)    540900      embedding_3[0][0]                
                                                                 input_1[0][0]                    
__________

In [50]:
decoder_model.fit(train_input, train_label, validation_data =(val_input, val_label), batch_size= 200, epochs=100, callbacks=callbacks_list)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [35]:
from random import randint
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    bleu_score = []
    result = list()
    in_text = seed_text
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
    # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    seq = ' '.join(result)
    bleu_score = sentence_bleu(seq, seed_text)
    return seq, bleu_score

In [36]:
#test 1
# load cleaned text sequences
in_filename = 'Balance sheet.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('GRU_1.hdf5')
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated, bleu = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

so given now we have more clarity with the tax reform do you think maybe for and beyond we might

see the industry that you re seeing in the fed book or is it a little bit on the balance sheet and i m curious if you could give us a little bit of the balance sheet and i m wondering you could have any bit more color in the


In [38]:
bleu

0.09752505550693362

In [44]:
#test 1
# load cleaned text sequences
in_filename = 'Balance sheet.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
# model = load_model('model.h6')

model.load_weights(filepath)
 
# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 30)
print(generated)

Initial text:
m just i guess wondering what the opportunity set for wells fargo could be if reg reform does come to

Generated text:
the industry that you ve been us in the fed hike to the industry that you re seeing more of the volatility of the industry that you ve been able


In [45]:
#test 2

# load the tokenizer
tokenizer = plk.load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print('Initial text:')
print(seed_text + '\n')
 
# generate new text
print('Generated text:')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

Initial text:
to deal with a negative line when things are going well and then on securitized products that were up what

Generated text:
the fed funds years i know you could give us a little bit on the industry that you ve been able to see the volatility of the industry that you re seeing more of the volatility of the industry that you re seeing in the fed book and then that


## Further improvement 

### 1. Improve the sequence build up based on each question, or expand training data to all questions from same category
### 2. Tune parameter to see if the performace is better in generating questions (batch size, epoch time etc.)
### 3. Research on the question generation process including tones and use of words
### 4. Improve the dropout (0.2, 0.4 0.6 etc.) to LSTM input layer to prevent overfitting for the model and may generate a more natural language

Dropout is a technique where randomly selected neurons are ignored during training. They are “dropped-out” randomly. This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass.
paper about dropout
http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf


